# Scrapeo de las competiciones de concurso completo

## Importación de librerías

In [3]:
# Importamos las librerías necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
from selenium.webdriver.support.ui import WebDriverWait
import xlrd
#from src.soporte_extraccion_general import get_competiciones, cambio_pestaña, resultados_disciplina, buscador_elementos, guardado_info, competiciones_año, obtencion_año, creacion_dictios_guardado
#from src.soporte_scrapeo_completo import extraccion_resultados_jinetes_caballos

In [2]:
# url inicial
url = "https://rfhe.com/competiciones/"

In [3]:
# inicializamos el driver y lo abrimos
driver = get_competiciones(url)

In [4]:
# creamos los diccionarios en los que vamos a ir guardando toda la información
dictio_concursos, dictio_pruebas, dictio_jinetes, dictio_caballos = creacion_dictios_guardado()

In [5]:
# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, disciplina = "completo")

In [6]:
ambito_buscado, disciplina_buscada

('nacional', 'completo')

In [7]:
# Buscamos los concursos de febrero del año que nos aparece
competiciones_año(driver)

In [8]:
# Sacamos el total de concursos que hay en el mes de marzo, que compondrá el total de veces que tendrá que iterar el bucle for siguiente
maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[2]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
maximo_n_concursos

3

In [9]:
# Buscamos el primer concurso del mes de marzo
i = 5
selector = f"/html/body/form/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[4]/a"
concurso = buscador_elementos(driver, selector).click()


In [10]:
# Sacamos el contenido general del concurso
contenido_general =  buscador_elementos(driver, "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
contenido_general

['Nombre: CCE CAVAHORSE- CCN 100-90-80-60',
 'Categoría: CCN 100',
 'Provincia: Girona',
 'Localidad: Ch Sant Marti Vell (Cava)',
 'Disciplina: Concurso Completo',
 'Fechas: 01 al 02 de Febrero de 2025',
 "Federación: Federació Catalana d'Hípica",
 'Avance: Disponible en PDF  ',
 '  Fechas incripción: 14/Ene->27/Ene/2025',
 'Inscripción: Concurso Finalizado',
 'Admitidos: Publicada  Ver',
 'Resultados: Ver resultados  Ver']

In [11]:
# Guardamos la información del concurso en el diccionario creado
guardado_info(dictio_concursos, contenido_general, info = "concursos", ambito = ambito_buscado)

In [12]:
dictio_concursos

{'Nombre': ['CCE CAVAHORSE- CCN 100-90-80-60'],
 'Categoría': ['CCN 100'],
 'Provincia': ['Girona'],
 'Localidad': ['Ch Sant Marti Vell (Cava)'],
 'Disciplina': ['Concurso Completo'],
 'Federación': ["Federació Catalana d'Hípica"],
 'Resultados': ['Ver'],
 'País': ['España']}

In [13]:
# Nos vamos a las pruebas del concurso
path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[20]/td[3]/a" 
pruebas = buscador_elementos(driver, path_boton_pruebas).click()
cambio_pestaña(2,driver)

In [14]:
# Obtenemos la información del concurso que se encuentra en la tabla de las pruebas
path_info_restante = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td"
info_concurso_restante = buscador_elementos(driver, path_info_restante).text.split("\n")

# Creamos las nuevas claves que sacamos de la info que esta donde las pruebas
dictio_concursos[info_concurso_restante[2].strip(":")] = []
dictio_concursos[info_concurso_restante[4].strip(":")] = []
dictio_concursos[info_concurso_restante[8].strip(":")] = []

In [15]:
# metemos la info del concurso que nos falta
guardado_info(diccionario = dictio_concursos, elementos = info_concurso_restante, claves = ["Inicio", "Final", "Ámbito"], indices = [3, 5, -1], info = "general")

In [16]:
dictio_concursos

{'Nombre': ['CCE CAVAHORSE- CCN 100-90-80-60'],
 'Categoría': ['CCN 100'],
 'Provincia': ['Girona'],
 'Localidad': ['Ch Sant Marti Vell (Cava)'],
 'Disciplina': ['Concurso Completo'],
 'Federación': ["Federació Catalana d'Hípica"],
 'Resultados': ['Ver'],
 'País': ['España'],
 'Inicio': ['01/02/2025'],
 'Final': ['02/02/2025'],
 'Ámbito': ['Nacional']}

In [17]:
# Obtenemos el nombre del concurso para luego meterlo en la tabla de pruebas
path_nombre_concurso = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td/div/div[1]/div/table/tbody/tr/td[2]/table/tbody/tr"
nombre_concurso = buscador_elementos(driver, path_nombre_concurso).text.strip()
nombre_concurso

'CCE CAVAHORSE- CCN 100-90-80-60'

In [18]:
# Obtenemos las pruebas del concurso
path_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]"
pruebas = buscador_elementos(driver, path_pruebas).text.split("\n")
pruebas


['Disciplina',
 'Fecha',
 'Prueba',
 'Categoría',
 'Número',
 'Resultados',
 'Concurso Completo',
 '01/02/2025',
 '100',
 'CCN 100',
 '1',
 'Disponibles',
 'Concurso Completo',
 '01/02/2025',
 '90',
 'CCN 90',
 '2',
 'Disponibles',
 'Concurso Completo',
 '01/02/2025',
 '80',
 'CCN 80',
 '3',
 'Disponibles',
 'Concurso Completo',
 '01/02/2025',
 '60',
 'CCN 60',
 '4',
 'Disponibles']

In [19]:
claves_pruebas = list(dictio_pruebas.keys())
guardado_info(diccionario = dictio_pruebas,elementos = pruebas, claves = claves_pruebas, indices = [6, 7, 8, 9, 10], step = 6, guardado = False, info = "general")

numero_pruebas = len(pruebas[6::6])  # Asumimos que 'Disciplina' tiene una fila por prueba
dictio_pruebas["Concurso"].extend([nombre_concurso] * numero_pruebas)

In [20]:
# accedemos a los resultados de la primera prueba
path_resultados_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]/div[2]/div/table/tbody/tr/td/div/div/table/tbody/tr[1]/td/div/div/table/tbody/tr[1]/td/div/div[3]/div/table/tbody/tr/td/a"
resultados_pruebas = buscador_elementos(driver, path_resultados_pruebas).click()
resultados_pruebas

In [21]:
cosas = pd.read_excel("../data/resultados_completo/RFHE_Resultados de la Prueba.xls", skiprows = 3)
cosas = cosas.rename(columns = {"Unnamed: 0": "Posicion", 
                                "LDN": "Lic.jinete",
                                "Nombre": "Jinete/Amazona", 
                                "LAC": "Lic.caballo", 
                                "Nombre.1": "Caballo",
                                "Unnamed: 6": "MER",
                                "Unnamed: 7": "Ptos_doma",
                                "Obs.": "Ptos_obs_cross",
                                "Tiempo": "Ptos_tmp_cross",
                                "Obs..1": "Ptos_obs_salto",
                                "Tiempo": "Ptos_tmp_salto",
                                "Unnamed: 12": "Total",
                                "Unnamed: 13": "Premio"})
cosas["Disciplina"] = "Completo"
cosas

,Posicion,Lic.jinete,Jinete/Amazona,Lic.caballo,Caballo,Raza,MER,Ptos_doma,Ptos_obs_cross,Ptos_tmp_salto,Ptos_obs_salto,Tiempo.1,Total,Premio,Disciplina
0,1,59612,David BECERRA MONTESINOS,163253,VELATINA EMDI,CDE,X,29.500000,0,0.000000,0,0.0,29.500000,0,Completo
1,2,36356,Carlota LUZON DE TENA,159852,ITS SO EASY GG,CDE,X,31.500000,0,0.000000,0,0.0,31.500000,0,Completo
2,3,212636,Lenon MAZER BARRIOS,130188,SIDRA G,CDE,X,32.799999,0,0.000000,0,0.0,32.799999,0,Completo
3,4,17237,Manuel SENRA CHOVER,156915,GALAAD DE BALLAC,SF,X,31.700001,0,1.600000,0,0.0,33.299999,0,Completo
4,5,15443,Carlos DÍAZ FERNÁNDEZ,169584,CONDOR HESA,CDE,X,33.400002,0,0.800000,0,0.0,34.200001,0,Completo
5,6,59612,David BECERRA MONTESINOS,146842,EA DE BARIAIN,(Indt.),X,34.599998,0,0.000000,0,0.0,34.599998,0,Completo
6,7,227930,Nahia FERNÁNDEZ FERRERAS,125043,HANS,CRUZ,X,34.700001,0,0.000000,0,0.0,34.700001,0,Completo
7,8,156950,Alvaro OLMEDILLA ARRIBAS,163263,HARMONY BK,CDE,X,36.099998,0,0.000000,0,0.0,36.099998,0,Completo
8,9,237254,Ricardo GÓMEZ VERDUGO,156600,HIPHOP DE LA MOULINE,SF,X,32.500000,0,3.600000,0,0.4,36.500000,0,Completo
9,10,16550,Gonzalo BLASCO BOTÍN,142416,KSANDRA,CDE,X,33.799999,0,2.400000,0,0.4,36.599998,0,Completo


In [22]:
extraccion_resultados_jinetes_caballos(driver, dictio_jinetes, dictio_caballos)

alert does not Exist in page
alert does not Exist in page
alert does not Exist in page
alert Exists in page


In [26]:
data_jinetes = pd.DataFrame(dictio_jinetes)
data_jinetes.to_csv("../data/resultados_completo/data_jinetes.csv", index=False)

In [28]:
data_jinetes

,Nombre,Licencia,Sexo,País,Federación,Disciplina
0,Cristian CABALLERO LÓPEZ,185106,Varón,ESP,Federació Catalana d'Hípica,Concurso Completo
1,Ariadna FARRÉ FARRÉS,271794,Mujer,ESP,Federació Catalana d'Hípica,Concurso Completo
2,Lessle PAMINA,323681,Mujer,ESP,Federació Catalana d'Hípica,Concurso Completo
3,Toni NAVARRO TRIOLA,271337,Varón,ESP,Federació Catalana d'Hípica,Concurso Completo
4,Laura BASCO MOLERO,182131,Mujer,ESP,Federació Catalana d'Hípica,Concurso Completo
5,Àlvar SALA ALONSO,206615,Varón,ESP,Federació Catalana d'Hípica,Concurso Completo
6,Laura FARRES CORELL,16452,Mujer,ESP,Federació Catalana d'Hípica,Concurso Completo
7,Aniol VILAREGUT PALAU,226221,Varón,ESP,Federació Catalana d'Hípica,Concurso Completo
8,Berta VALLS LOP,302746,Mujer,ESP,Federació Catalana d'Hípica,Concurso Completo
9,Edith PORCEL FOLLET,189309,Mujer,ESP,Federació Catalana d'Hípica,Concurso Completo


In [27]:
data_caballos = pd.DataFrame(dictio_caballos)
data_caballos.to_csv("../data/resultados_completo/data_caballos.csv", index=False)

In [29]:
data_caballos

,Nombre,Licencia,Edad,País,Raza,Sexo,Federación,Disciplina
0,HYDE PARK,164061,13,ESP,K.W.P.N.,Castrado,Federación:,Concurso Completo
1,JOR-DINA,158583,13,ESP,Árabe,Hembra,Federación:,Concurso Completo
2,MISTERIOSO DE CRIS,155182,7,ESP,Pura Raza Español,Castrado,Federación:,Concurso Completo
3,SUFRIA,154978,10,ESP,Cruzado,Hembra,Federación:,Concurso Completo
4,"ESLA, AA 35,94%",163254,5,ESP,Anglo Árabe,Hembra,Federación:,Concurso Completo
5,BIG BOY CHT,167886,5,ESP,Cab. Deporte Español,Castrado,Federación:,Concurso Completo
6,TOO MUCH LF,168854,5,ESP,Cab. Deporte Español,Castrado,Federación:,Concurso Completo
7,GINEBRA DE VIPA,159965,11,ESP,Cab. Deporte Español,Hembra,Federación:,Concurso Completo
8,ORGULLOSA 28.14%,168139,18,ESP,Anglo Árabe,Hembra,Federación:,Concurso Completo
9,PEQUENA,165805,6,ESP,Indeterminada,Hembra,Federación:,Concurso Completo


## Concursos nacionales

In [ ]:
# url inicial
url = "https://rfhe.com/competiciones/"

dictio_concursos_completo_nac, dictio_pruebas_completo_nac, dictio_jinetes_completo_nac, dictio_caballos_completo_nac = creacion_dictios_guardado()
urls_resultados = []

driver = get_competiciones(url)
time.sleep(2)

# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, disciplina = "completo")
cambio_pestaña(1,driver)
time.sleep(2)

año = obtencion_año(driver)

while año >= 2017:

    # vamos a todos los concursos del año en el que estemos
    competiciones_año(driver)

    maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[13]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
    if año == 2025:
        rango = range(5,21)
    elif año < 2025:
        rango = range(5, maximo_n_concursos + 5) 

    for i in rango:

        try:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/a")
            print(concurso_bueno.text)
            time.sleep(1)
            try:
                concurso_bueno.click()
                cambio_pestaña(1, driver)
                time.sleep(2)
                contenido_general =  buscador_elementos(driver, "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
                                                                
                if "Resultados: Ver resultados  Ver" not in contenido_general:
                    driver.back()
                    time.sleep(1)

                elif "Resultados: Ver resultados  Ver" in contenido_general:
                    print(contenido_general)
                    
                    extraccion_info_concursos(driver, diccionario_concursos=dictio_concursos_completo_nac, ambito_buscado=ambito_buscado, contenido_general=contenido_general)
                    cambio_pestaña(2, driver)
                    time.sleep(1)

                    if i == 5 and año == 2025:
                        extraccion_info_pruebas(driver, dictio_concursos_completo_nac, dictio_pruebas_completo_nac, urls_resultados, es_primer_concurso=True)
                    else:
                        extraccion_info_pruebas(driver, dictio_concursos_completo_nac, dictio_pruebas_completo_nac, urls_resultados)

                    driver.close()
                    cambio_pestaña(1, driver)
                    driver.back()
                    time.sleep(2)

            except NoSuchElementException:
                raise NoSuchElementException

        except NoSuchElementException:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/font").text
            print(concurso_bueno)
    
    buscador_elementos(driver,"/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/a").click()        
    año = obtencion_año(driver)

## Concursos internacionales

In [ ]:
# url inicial
url = "https://rfhe.com/competiciones/"

dictio_concursos_completo_int, dictio_pruebas_completo_int, dictio_jinetes_completo_int, dictio_caballos_completo_int = creacion_dictios_guardado()
urls_resultados_int = []

# inicializamos el driver y lo abrimos
driver = get_competiciones(url)
time.sleep(2)

# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, ambito = "internacional", disciplina = "completo")
cambio_pestaña(1,driver)
time.sleep(2)

año = obtencion_año(driver)

while año >= 2024:

    # Buscamos los concursos del año que nos aparece
    competiciones_año(driver)

    time.sleep(3)

    maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[13]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
    if año == 2025:
        rango = range(5,33)
    elif año < 2025:
        rango = range(5, maximo_n_concursos + 5) 

    for i in rango:
        
        try:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/a")
            print(concurso_bueno.text)
            time.sleep(1)
            try:
                concurso_bueno.click()
                cambio_pestaña(2, driver)
                time.sleep(2)
                contenido_general = buscador_elementos(driver, "/html/body/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
                                                                
                if "Resultados: Ver resultados  Ver" not in contenido_general:
                    driver.close()
                    cambio_pestaña(1, driver)
                    time.sleep(1) 
                
                elif "Resultados: Ver resultados  Ver" in contenido_general:
                    extraccion_info_concursos(driver, diccionario_concursos=dictio_concursos_completo_int, ambito_buscado=ambito_buscado, contenido_general=contenido_general)
                    cambio_pestaña(3,driver)
                    if i == 5 and año == 2025:
                        extraccion_info_pruebas(driver, dictio_concursos_completo_int, dictio_pruebas_completo_int, urls_resultados_int, es_primer_concurso=True)
                    else:
                        extraccion_info_pruebas(driver, dictio_concursos_completo_int, dictio_pruebas_completo_int, urls_resultados_int)
                    driver.close()
                    cambio_pestaña(2, driver)
                    driver.close()
                    cambio_pestaña(1, driver)
                    time.sleep(2)

            except NoSuchElementException:
                raise NoSuchElementException

        except NoSuchElementException:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/font").text
            print(concurso_bueno)
    
    buscador_elementos(driver,"/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/a").click()
    time.sleep(3)        
    año = obtencion_año(driver)
driver.quit()

## Comprobaciones scrapeo

In [2]:
import json

In [ ]:
# Abrir y cargar un archivo JSON
with open('data/data_completo/concursos/concursos_completo_internacional_2023.json', 'r', encoding='utf-8') as f:
    datos = json.load(f)

datos

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_completo/concursos/concursos_completo_internacional_2023.json'

In [10]:
from seleniumbase import Driver

In [9]:
%pip install seleniumbase

  Using cached seleniumbase-4.38.0-py3-none-any.whl.metadata (86 kB)
  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached setuptools-80.3.1-py3-none-any.whl.metadata (6.5 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fasteners-0.19-py3-none-any.whl.metadata (4.9 kB)
  Using cached mycdp-1.2.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached pynose-1.5.4-py3-none-any.whl.metadata (19 kB)
  Using cached sbvirtualdisplay-1.4.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached parse-1.20.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached parse_type-0.6.4-py2.py3-none-any.whl.metad

In [5]:
cosita = pd.read_excel("../data/data_completo/resultados/RFHE_Resultados de la Prueba_2023.xls")
cosita

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Posición,Jinete/Amazona,NaN,Caballo,NaN,NaN,MER,Doma,Cross,NaN,Salto,NaN,Ptos.,Premios
2,NaN,FEI,Nombre,FEI,Nombre,Raza,NaN,NaN,Obs.,Tiempo,Obs.,Tiempo,NaN,NaN
3,1,10004636,Karim Florent LAGHOUAG (FRA),106WL31,DREAM DE VIEVE (FRA),--,X,0,0,0,0,108.790001,0,4000
4,2,10070530,Fabrice SAINTEMARIE (FRA),105WU66,CESAR DE ROY (FRA),--,X,0,0,0,0,110.339996,0,3000
5,3,10044029,Benjamin MASSIE (FRA),106JW63,CUPIDON DU CARDONNE (FRA),--,X,0,0,0,0,113.260002,0,2250
6,4,10130945,Salome POISSON (FRA),104GE91,UPSA DES AJONCS (FRA),--,X,0,0,0,0,114,0,1500
7,5,10089941,Korntawat SAMRAN (THA),104KU05,USTER DE CHANAY,SF,X,0,0,0,0,114.830002,0,1000
8,6,10060608,Luc CHATEAU (FRA),104TN63,TROUBADOUR CAMPHOUX (FRA),--,X,0,0,0,0,116.830002,0,800
9,7,10119172,Julie SIMONET (FRA),103RK33,SURSUMCORD'OR (FRA),--,X,0,0,0,0,117.849998,0,600
